In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:

data = pd.read_csv('MASTER_MSFT_FINAL.csv',index_col=[0])
stock_data = pd.read_csv('MSFT_STOCK.csv',index_col=[0])

In [5]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,MA7,MA20,MACD,...,title,web,author,combined_news,Subjectivity,Polarity,compound,neg,pos,neu
6193,2024-08-26,416.369995,417.279999,411.339996,413.489990,413.489990,13152800,419.252856,413.112497,2.295145,...,['EDP Collaborates with Microsoft on Largest S...,"['https://www.edpr.com', 'https://www.bleeping...","['EDP Renewables', 'BleepingComputer', 'Market...",EDP Collaborates with Microsoft on Largest Sol...,0.445177,0.153483,0.5345,0.070,0.068,0.862
6194,2024-08-27,412.859985,414.359985,410.250000,413.839996,413.839996,13492900,418.591426,412.658496,2.612213,...,"[""Super Micro Computer Stock Hit by Report All...","['https://www.investopedia.com', 'https://carb...","['Investopedia', 'CarbonCredits.com', 'Fortune...",Super Micro Computer Stock Hit by Report Alleg...,0.495721,0.197389,0.9610,0.049,0.070,0.881
6195,2024-08-28,414.880005,415.000000,407.309998,410.600006,410.600006,14882700,417.029999,412.270996,2.291256,...,['Nvidia stock could plunge 20% if Big Tech st...,"['https://www.cnbc.com', 'https://www.statista...","['CNBC', 'Statista', ""Investor's Business Dail...",Nvidia stock could plunge 20% if Big Tech star...,0.416149,0.119387,0.9823,0.068,0.091,0.841
6196,2024-08-29,414.940002,422.049988,410.600006,413.119995,413.119995,17045200,415.361428,412.071497,2.142343,...,"[""The bond market is about to do something not...","['https://www.cnbc.com', 'https://www.thestree...","['CNBC', 'TheStreet', 'The Motley Fool', 'Inve...",The bond market is about to do something notew...,0.482179,0.234667,-0.6232,0.067,0.065,0.868
6197,2024-08-30,415.600006,417.489990,412.130005,417.140015,417.140015,24298300,414.361428,412.503998,2.176759,...,"['Is Lucid Stock Doomed? - Energy & Capital', ...","['https://www.energyandcapital.com', 'https://...","['Energy & Capital', 'Financial Times', 'Inves...",Is Lucid Stock Doomed? - Energy & Capital Soth...,0.466567,0.174304,0.9597,0.081,0.096,0.822


In [10]:
stock_data = stock_data.reset_index()

In [42]:
# Convert date to datetime format
data['date'] = pd.to_datetime(data['Date'])
stock_data['date'] = pd.to_datetime(stock_data['Date'])

In [43]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all special characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.lower()  # Convert to lowercase
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

data['cleaned_headline'] = data['combined_news'].apply(clean_text)

In [44]:
# Use TF-IDF Vectorizer to prepare for LDA
# 
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_headline'])

In [45]:
# Apply LDA to the TF-IDF matrix
n_topics = 5  # You can adjust this number based on experimentation
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_topics = lda_model.fit_transform(tfidf_matrix)

# Get the top words for each topic
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(lda_model, tfidf_vectorizer.get_feature_names_out(), 10)

Topic 0:
newsround cbbc bbc washington news post cnn market business 2003
Topic 1:
knowledge fox thestreet daily atlanta democrat china list mark project
Topic 2:
2000 cnn 2006 aug jan mar 17 16 14 19
Topic 3:
market microsoft stock new news com time window business xbox
Topic 4:
stock microsoft market finance yahoo fool motley cnbc new investor


In [46]:
# Add topic distributions back to the original dataframe
topic_columns = [f"topic_{i}" for i in range(n_topics)]
topic_df = pd.DataFrame(lda_topics, columns=topic_columns)
data = pd.concat([data, topic_df], axis=1)

# Aggregate by month or quarter (here we do monthly aggregation)
data['month'] = data['date'].dt.to_period('M')
monthly_topic_trends = data.groupby('month')[topic_columns].mean().reset_index()